In [1]:
using PyCall

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/PyCall.ji for module PyCall.


In [27]:
using Plots
gr()

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/FixedSizeArrays.ji for module FixedSizeArrays.
INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/PlotThemes.ji for module PlotThemes.
INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/Showoff.ji for module Showoff.
INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/GR.ji for module GR.


Plots.GRBackend()

In [2]:
pyimport_conda("sklearn", "scikit-learn")

INFO: Installing sklearn via the Conda scikit-learn package...


Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /Users/rdeits/.julia/v0.5/Conda/deps/usr:

The following NEW packages will be INSTALLED:

    scikit-learn: 0.18.1-np112py27_1

scikit-learn-0 100% |###############################| Time: 0:00:00   6.58 MB/s


PyObject <module 'sklearn' from '/Users/rdeits/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/sklearn/__init__.pyc'>

In [3]:
@pyimport sklearn.svm as svm

In [7]:
include("main.jl")

c

In [40]:
double_integrator = c.MPC.discretize(c.MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)
results = c.collect_data(double_integrator, 10000);

In [41]:
examples = convert(Vector{c.Example}, results);

In [42]:
labels = [e.best_action for e in examples]
data = hcat([vcat(e.contact, e.duals) for e in examples]...);

In [43]:
clf = svm.SVC(C=100)
clf[:fit](data[:, 1:2:end]', labels[1:2:end])
predicted_labels = clf[:predict](data[:, 2:2:end]');

mean((predicted_labels .== labels[2:2:end]))*100

61.28

In [44]:
function accuracy(C)
    clf = svm.SVC(C=C)
    clf[:fit](data[:, 1:2:end]', labels[1:2:end])
    predicted_labels = clf[:predict](data[:, 2:2:end]');
    mean((predicted_labels .== labels[2:2:end]))
end

accuracy (generic function with 2 methods)

In [45]:
Cs = logspace(-1, 5, 20);
a = accuracy.(Cs);

plot(Cs, a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25000 
 
 
 50000 
 
 
 75000 
 
 
 100000 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 
 
 
 
 y1